In [3]:
import requests
import geopandas as gpd

In [4]:
class IsochroneService():

    def __init__(
        self, 
        times = None, 
        reference_date = '02-24-2022',
        walk_reluctance = 5,
        max_walk_distance = 1500,
        min_transfer_time = 120,
        mode = 'WALK,TRANSIT',
        otp_port = 8801,
    ):
        if times is None:
            times = {'morning': "08:00am", 'afternoon': "2:00pm", 'evening': "8:00pm"}
        self.times = times
        self.ref_date = reference_date
        self.walk_reluctance = walk_reluctance
        self.min_transfer_time = min_transfer_time
        self.mode = mode
        self.max_walk_distance = max_walk_distance
        self.otp_port = otp_port
    

    def get_isochrone(self, lat, lon, city='atlanta', time='morning', minutes=30):
        #documentation @ http://dev.opentripplanner.org/apidoc/1.5.0/resource_LIsochrone.html
        
        assert time in self.times, "Invalid time category. Provided {}, should be one of {}".format(time, ", ".join(self.times))
        time_spec = self.times[time]

        response = requests.get(
            'http://localhost:{}/otp/routers/{}/isochrone'.format(self.otp_port, city),
            params= {
                'toPlace': '{},{}'.format(lat, lon), #lat long pair of starting point
                'mode': self.mode,
                'date': self.ref_date,
                'time': time_spec,
                'maxWalkDistance': self.max_walk_distance,
                'walkReluctance': self.walk_reluctance,
                'minTransferTime': self.min_transfer_time,
                'cutoffSec': minutes * 60
            },
        )
        
        return gpd.read_file(response.text).explode(index_parts=True)

In [5]:
service = IsochroneService()
service.get_isochrone(33.781961, -84.396635)

DriverError: java.lang.NullPointerException null: No such file or directory